In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sentiment_functions import score_flair, get_news, get_gnews_article, get_sentiments, getTwitterData,get_flair_score

In [18]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=100,
        max_font_size=30,
        scale=3,
        random_state=1)
   
    wordcloud=wordcloud.generate(str(data))

    fig = plt.figure(1, figsize=(12, 12))
    plt.axis('off')

    plt.imshow(wordcloud)
    plt.show()


In [23]:
def get_all_sentiments(key, no_days):
    
    # A list of dates from today going back 'no_days' in ascending order
    datelist = pd.bdate_range(end = dt.date.today(), periods = int(no_days))

    summary_df = []

    # iterate through the list of dates to get the headlines for each date
    for date in datelist:
        df = pd.DataFrame(get_news(key, date))
        if(len(df) > 20):
            article_df = get_gnews_article(df[:20]) 
        else:
            article_df = get_gnews_article(df)
        summary_df.append(article_df)
    
    df2 = pd.concat(summary_df).set_index('Date')
    df2 = df2.dropna()
    df2 = df2.reset_index()
    
    #show_wordcloud(df2['Key_words'])
    
    dic = {}
    for i in range(len(df2)):
        date = df2['Date'][i]
        summary = df2['Article'][i]
        if date in dic:
            dic[date].append(summary)
        else:
            dic[date] = [summary]

    news_df = pd.DataFrame.from_dict(dic, orient = 'index')
    news_df = news_df.reset_index()
    news_df['index'] = pd.to_datetime(news_df['index'])
    
    news_df = news_df.replace(np.nan, 'none')
    news_df = news_df.replace('', 'none')

    news_sentiment = get_sentiments(news_df)
    
    tweets_df = getTwitterData(key, no_days) 
    tweets_df = tweets_df.reset_index()
    
    tweet_dic = {}
    for i in range(len(tweets_df)):
        date = tweets_df['Date'][i]
        summary = tweets_df['full_text'][i]
        if date in tweet_dic:
            tweet_dic[date].append(summary)
        else:
            tweet_dic[date] = [summary]
        
    twitter_df = pd.DataFrame.from_dict(tweet_dic, orient = 'index')
    twitter_df = twitter_df.reset_index()
    twitter_df['index'] = pd.to_datetime(twitter_df['index'])
    
    twitter_df = twitter_df.replace(np.nan, 'none')
    twitter_df = twitter_df.replace('', 'none')
    
    
    
    tweets_sentiment = get_sentiments(twitter_df)    
    tweets_sentiment.columns = ['index', 'tw_subj_score', 'tw_simi_score', 'tw_vader_score']
    
    flair_list = get_flair_score(twitter_df)
    tweets_sentiment['tw_scores_flair'] = flair_list
    
    sentiment_df = news_sentiment.join(tweets_sentiment.set_index('index'), on = 'index')
    sentiment_df = sentiment_df.dropna()
    return sentiment_df
    

In [24]:
s_df = get_all_sentiments('tsla', 5)


C:\Users\abhis\Anaconda3\envs\project2-dev\lib\site-packages\dateparser\date_parser.py:37: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\abhis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\abhis\OneDrive\Desktop\Fintech-Workspace\Project_2\sentiment_functions.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [25]:
s_df

,index,subj_score,simi_score,vader_score,tw_subj_score,tw_simi_score,tw_vader_score,tw_scores_flair
0,2022-01-12,0.345408,0.697772,0.820365,0.100238,0.189538,0.215292,0.919249
1,2022-01-13,0.407614,0.933854,0.923205,0.110363,0.103158,0.243046,0.874333
2,2022-01-14,0.376957,0.897357,0.905400,0.031625,-0.042091,0.077835,0.774399
3,2022-01-17,0.335855,0.777622,0.830875,0.067422,0.113611,0.210635,0.851678
